# BERT-language2label

In [1]:
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt/code")' Batcher ModelSupport Datasets TextModels

import Datasets
import Foundation
import ModelSupport
import TensorFlow
import TextModels
import PythonKit

Installing packages:
	.package(path: "/notebooks/language2motion.gt/code")
		Batcher
		ModelSupport
		Datasets
		TextModels
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpebh7jqf8/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
Initializing Swift...
Installation complete!


In [2]:
let bertPretrained = BERT.PreTrainedModel.bertBase(cased: false, multilingual: false)
let workspaceURL = URL(
    fileURLWithPath: "bert_models", isDirectory: true,
    relativeTo: URL(
        fileURLWithPath: NSTemporaryDirectory(),
        isDirectory: true))
let bert = try BERT.PreTrainedModel.load(bertPretrained)(from: workspaceURL)
var bertClassifier = BERTClassifier(bert: bert, classCount: 5)

Loading BERT pre-trained model 'BERT Base Uncased'.
Loading resource: uncased_L-12_H-768_A-12


## load dataset

In [3]:
// TODO: 
// + load csv
// + split into train/dev
// + convert to [Example]
// + get sorted labels
// + integrate with Language2Label
// - Language2Label code cleanups
// - extraxt Language2Label codes
// * train

In [4]:
let pd  = Python.import("pandas")
let model_selection  = Python.import("sklearn.model_selection")

In [5]:
public struct Language2Label {
//     public let directoryURL: URL
    public let trainExamples: [Example]
    public let devExamples: [Example]
    public let maxSequenceLength: Int
    public let batchSize: Int
    public let labels: [String]

    public typealias ExampleIterator = IndexingIterator<[Example]>
    public typealias TrainDataIterator = PrefetchIterator<
        GroupedIterator<MapIterator<ExampleIterator, DataBatch>>
    >
    public typealias DevDataIterator = GroupedIterator<MapIterator<ExampleIterator, DataBatch>>
    public typealias TestDataIterator = DevDataIterator

    public var trainDataIterator: TrainDataIterator
    public var devDataIterator: DevDataIterator
}

//===-----------------------------------------------------------------------------------------===//
// Data
//===-----------------------------------------------------------------------------------------===//

extension Language2Label {
    /// Language2Label example.
    public  struct Example {
        public typealias LabelTuple = (idx: Int, label: String)

        public let id: String
        public let text: String
        public let label: LabelTuple?

        public init(id: String, text: String, label: LabelTuple?) {
            self.id = id
            self.text = text
            self.label = label
        }
    }
    /// Language2Label data batch.
    public struct DataBatch: KeyPathIterable {
        public var inputs: TextBatch  // TODO: !!! Mutable in order to allow for batching.
        public var labels: Tensor<Int32>?  // TODO: !!! Mutable in order to allow for batching.

        public init(inputs: TextBatch, labels: Tensor<Int32>?) {
            self.inputs = inputs
            self.labels = labels
        }
    }

    /// URL pointing to the downloadable ZIP file that contains the CoLA dataset.
//     private static let url: URL = URL(
//         string: String(
//             "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/"
//                 + "o/data%2FCoLA.zip?alt=media&token=46d5e637-3411-4188-bc44-5809b5bfb5f4"))!

    internal enum FileType: String {
        case train = "train"
        case dev = "dev"
        case test = "test" // TODO: kill
    }
//     internal static func load(fromFile fileURL: URL, fileType: FileType) throws -> [Example] {
//         let lines = try parse(tsvFileAt: fileURL)

//         if fileType == .test {
//             // The test data file has a header.
//             return lines.dropFirst().enumerated().map { (i, lineParts) in
//                 Example(id: lineParts[0], sentence: lineParts[1], isAcceptable: nil)
//             }
//         }

//         return lines.enumerated().map { (i, lineParts) in
//             Example(id: lineParts[0], sentence: lineParts[3], isAcceptable: lineParts[1] == "1")
//         }
//     }
    
    static func Df2Example(df: PythonObject, labels: [String]) -> [Example] {
        return Python.list(df.iterrows()).map {
            (rowObj: PythonObject) -> Example in 
            let row = rowObj.tuple2.1
            let sample_id: String = "\(row.sample_id)" // Int to String
            let text: String = String(row.text)!
            let labelStr: String? = String(row.label)
            let label: Example.LabelTuple? = Example.LabelTuple(idx: labels.index(of: labelStr!)!, label: labelStr!)
            return Example(id: sample_id, text: text, label: label)
        }
    }
}

// internal func parse(tsvFileAt fileURL: URL) throws -> [[String]] {
//     try Data(contentsOf: fileURL).withUnsafeBytes {
//         $0.split(separator: UInt8(ascii: "\n")).map {
//             $0.split(separator: UInt8(ascii: "\t"), omittingEmptySubsequences: false)
//                 .map { String(decoding: UnsafeRawBufferPointer(rebasing: $0), as: UTF8.self) }
//         }
//     }
// }

extension Language2Label {
    public init(
        exampleMap: @escaping (Example) -> DataBatch,
        taskDirectoryURL: URL,
        maxSequenceLength: Int,
        batchSize: Int,
        dropRemainder: Bool
    ) throws {
//         self.directoryURL = taskDirectoryURL.appendingPathComponent("CoLA")
//         let dataURL = directoryURL.appendingPathComponent("data")
//         let compressedDataURL = dataURL.appendingPathComponent("downloaded-data.zip")

//         // Download the data, if necessary.
//         try download(from: Language2Label.url, to: compressedDataURL)

//         // Extract the data, if necessary.
//         let extractedDirectoryURL = compressedDataURL.deletingPathExtension()
//         if !FileManager.default.fileExists(atPath: extractedDirectoryURL.path) {
//             try extract(zipFileAt: compressedDataURL, to: extractedDirectoryURL)
//         }

//         #if false
//             // FIXME: Need to generalize `DatasetUtilities.downloadResource` to accept
//             // arbitrary full URLs instead of constructing full URL from filename and
//             // file extension.
//             DatasetUtilities.downloadResource(
//                 filename: "\(subDirectory)", fileExtension: "zip",
//                 remoteRoot: url.deletingLastPathComponent(),
//                 localStorageDirectory: directory)
//         #endif

        let dsURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/labels.csv")
        let df = pd.read_csv(dsURL.path)
        labels = df.label.unique().sorted().map {String($0)!}
        let (train_df, test_df) = model_selection.train_test_split(df, test_size: 0.2).tuple2
        
        trainExamples = Language2Label.Df2Example(df: train_df, labels: labels)
        devExamples = Language2Label.Df2Example(df: test_df, labels: labels)        

        self.maxSequenceLength = maxSequenceLength
        self.batchSize = batchSize

        // Create the data iterators used for training and evaluating.
        self.trainDataIterator = trainExamples.shuffled().makeIterator()  // TODO: [RNG] Seed support.
            .map(exampleMap)
            .grouped(
                keyFn: { _ in 0 },
                sizeFn: { _ in batchSize / maxSequenceLength },
                reduceFn: {
                    DataBatch(
                        inputs: padAndBatch(
                            textBatches: $0.map { $0.inputs }, maxLength: maxSequenceLength),
                        labels: Tensor.batch($0.map { $0.labels! }))
                },
                dropRemainder: dropRemainder
            )
            .prefetched(count: 2)
        self.devDataIterator = devExamples.makeIterator()
            .map(exampleMap)
            .grouped(
                keyFn: { _ in 0 },
                sizeFn: { _ in batchSize / maxSequenceLength },
                reduceFn: {
                    DataBatch(
                        inputs: padAndBatch(
                            textBatches: $0.map { $0.inputs }, maxLength: maxSequenceLength),
                        labels: Tensor.batch($0.map { $0.labels! }))
                },
                dropRemainder: dropRemainder
            )
    }
}

In [6]:
// Regarding the batch size, note that the way batching is performed currently is that we bucket
// input sequences based on their length (e.g., first bucket contains sequences of length 1 to 10,
// second 11 to 20, etc.). We then keep processing examples in the input data pipeline until a
// bucket contains enough sequences to form a batch. The batch size specified in the task
// constructor specifies the *total number of tokens in the batch* and not the total number of
// sequences. So, if the batch size is set to 1024, the first bucket (i.e., lengths 1 to 10)
// will need 1024 / 10 = 102 examples to form a batch (every sentence in the bucket is padded
// to the max length of the bucket). This kind of bucketing is common practice with NLP models and
// it is done to improve memory usage and computational efficiency when dealing with sequences of
// varied lengths. Note that this is not used in the original BERT implementation released by
// Google and so the batch size setting here is expected to differ from that one.
let maxSequenceLength = 128
let batchSize = 1024

// Create a function that converts examples to data batches.
let exampleMapFn: (Language2Label.Example) -> Language2Label.DataBatch = { example -> Language2Label.DataBatch in
    let textBatch = bertClassifier.bert.preprocess(
        sequences: [example.text],
        maxSequenceLength: maxSequenceLength)
    return Language2Label.DataBatch( // TODO: get label idx
        inputs: textBatch, labels: example.label.map { (label: Language2Label.Example.LabelTuple) in Tensor( Int32(label.idx)) })
}

var dataset = try Language2Label(
    exampleMap: exampleMapFn,
    taskDirectoryURL: workspaceURL,
    maxSequenceLength: maxSequenceLength,
    batchSize: batchSize,
    dropRemainder: true)

print("Dataset acquired.")

Dataset acquired.


In [7]:
dataset.trainExamples.count

2409


In [8]:
dataset.trainExamples[0]

▿ Example
  - id : "1451"
  - text : "A person lifts up his left arm, bent at the elbow, then moves his right arm across his chest, hand just above the elbow of the raised left arm."
  ▿ label : Optional<(idx: Int, label: String)>
    ▿ some : 2 elements
      - idx : 1
      - label : "Performing motions with hands"


In [9]:
dataset.devExamples[0]

▿ Example
  - id : "1982"
  - text : "A person walks a quarter circle counter clockwise with 4 steps."
  ▿ label : Optional<(idx: Int, label: String)>
    ▿ some : 2 elements
      - idx : 2
      - label : "Walking and turning"


## train

In [10]:
var optimizer = WeightDecayedAdam(
    for: bertClassifier,
    learningRate: LinearlyDecayedParameter(
        baseParameter: LinearlyWarmedUpParameter(
            baseParameter: FixedParameter<Float>(2e-5),
            warmUpStepCount: 10,
            warmUpOffset: 0),
        slope: -5e-7,  // The LR decays linearly to zero in 100 steps.
        startStep: 10),
    weightDecayRate: 0.01,
    maxGradientGlobalNorm: 1)

In [11]:
print("Training BERT for the Language2Label task!")
for epoch in 1...3 {
    print("[Epoch \(epoch)]")
    Context.local.learningPhase = .training
    var trainingLossSum: Float = 0
    var trainingBatchCount = 0
    var trainingDataIterator = dataset.trainDataIterator
    
    while let batch = withDevice(.cpu, perform: { trainingDataIterator.next() }) {
        let (documents, labels) = (batch.inputs, Tensor<Int32>(batch.labels!))
        let (loss, gradients) = valueWithGradient(at: bertClassifier) { model -> Tensor<Float> in
            let logits = model(documents)
                                                                    
            return softmaxCrossEntropy(logits: logits, labels: labels)
        }

        trainingLossSum += loss.scalarized()
        trainingBatchCount += 1
        optimizer.update(&bertClassifier, along: gradients)

        print(
            """
              Training loss: \(trainingLossSum / Float(trainingBatchCount))
            """
        )
    }

    Context.local.learningPhase = .inference
    var devLossSum: Float = 0
    var devBatchCount = 0
    var devDataIterator = dataset.devDataIterator
    var devPredictedLabels = [Bool]()
    var devGroundTruth = [Bool]()
    while let batch = withDevice(.cpu, perform: { devDataIterator.next() }) {
        print("test batch")
        let (documents, labels) = (batch.inputs, batch.labels!)
        let logits = bertClassifier(documents)
        let loss = sigmoidCrossEntropy(
            logits: logits.squeezingShape(at: -1),
            labels: Tensor<Float>(labels),
            reduction: { $0.mean() }
        )
        devLossSum += loss.scalarized()
        devBatchCount += 1

        let predictedLabels = sigmoid(logits.squeezingShape(at: -1)) .>= 0.5
        devPredictedLabels.append(contentsOf: predictedLabels.scalars)
        devGroundTruth.append(contentsOf: labels.scalars.map { $0 == 1 })
    }
    print(4)

    let mcc = matthewsCorrelationCoefficient(
        predictions: devPredictedLabels,
        groundTruth: devGroundTruth)

    print(
        """
          MCC: \(mcc)
          Eval loss: \(devLossSum / Float(devBatchCount))
        """
    )
}

Training BERT for the Language2Label task!
[Epoch 1]
  Training loss: 1.5286338
  Training loss: 1.6280258
  Training loss: 1.7589718
  Training loss: 1.7574695
  Training loss: 1.8055675
  Training loss: 1.798664
  Training loss: 1.8284141
  Training loss: 1.8270429
  Training loss: 1.8305086
  Training loss: 1.7910993
  Training loss: 1.7708272
  Training loss: 1.7626184
  Training loss: 1.7480073
  Training loss: 1.7085618
  Training loss: 1.679118
  Training loss: 1.6711938
  Training loss: 1.6812017
  Training loss: 1.6623082
  Training loss: 1.6307421
  Training loss: 1.6271813
  Training loss: 1.6146624
  Training loss: 1.6112064
  Training loss: 1.5804316
  Training loss: 1.5612737
  Training loss: 1.5464345
  Training loss: 1.523027
  Training loss: 1.5425303
  Training loss: 1.5300928
  Training loss: 1.5282929
  Training loss: 1.534577
  Training loss: 1.5237814
  Training loss: 1.5066097
  Training loss: 1.4923956
  Training loss: 1.5102407
  Training loss: 1.4931664
  Trai

: 